In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Download and unzip data

In [ ]:
!cp /content/drive/MyDrive/files/data_path.zip /content
!unzip data_path.zip "data/jsons/*" "data/pic/*" "data/ques_embeddings/bioelmo/*" "data/answer_word_frequency.csv" "data/answer_word_frequency.xlsx" -d /content

In [ ]:
!mkdir data
!cp /content/drive/MyDrive/data/jsons/QA.json /content/data
!cp /content/drive/MyDrive/data/knowledge.txt /content/data

mkdir: cannot create directory ‘data’: File exists


In [ ]:
!cat /content/drive/MyDrive/data/knowledge.txt

# Entity extraction

# Knowledge Embeddings

In [ ]:
pip install transformers

     |████████████████████████████████| 2.3MB 2.9MB/s 
     |████████████████████████████████| 901kB 13.1MB/s 
     |████████████████████████████████| 3.3MB 17.6MB/s 


In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
import numpy as np
import json
import re
import os

tokenizer = AutoTokenizer.from_pretrained("bert-large-cased", do_lower_case=True)
model = AutoModel.from_pretrained("bert-large-cased", output_hidden_states=True)

with open('/content/drive/MyDrive/data/knowledge_slakekg.txt','r') as file:
    lines = file.readlines()

path = '/content/knowledge_embeddings'
if not os.path.exists(path):
    os.mkdir(path)


Some weights of the model checkpoint at bert-large-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
kns = []
for l in lines:
  l_split = l.strip().replace('#',' ').split()
  kns.append((l_split[0],' '.join(l_split[1:61])))
print(len(kns))

32761


In [ ]:
print(lines[6998])
print(kns[6998])

7006,hyperopia description hyperopia is a refractive state in which parallel beams of light are imaged behind the retina after adjusting the relaxed refraction of the eye, producing hyperopia when the refraction of the eye is insufficient or when the axial length of the eye is insufficient.,hemangioma description hemangiomas are congenital benign tumors or vascular malformations that arise from the proliferation of angioblasts during the embryonic period, often in the skin and soft tissues, and often occur at or shortly after the birth of an infant.,cerebral palsy syndrome symptom dyskinesia, abnormal posture,hydrocephalus symptom pain, nausea, dyskinesia,patella fracture description patella fracture is a common injury, characterized by local patellar swelling, pain, inability to extend the knee spontaneously, subcutaneous ecchymosis and skin abrasion of the knee.,syringomyelia symptom segmental sensory disturbance, limb dyskinesia,spinal fracture symptom localized swelling, tenderness

In [ ]:
for kn in kns:
  ques_id = int(kn[0])
  k = kn[1]
  # k_emb = []
  # for k in ks:
  k = '[CLS] ' + k + ' [SEP]'
  tokenized_text1 = tokenizer.tokenize(k) 
  # print(len(tokenized_text1))
  indexed_tokens1 = tokenizer.convert_tokens_to_ids(tokenized_text1)    
  segments_ids1 = [1] * len(tokenized_text1)  
  # shape: (1,17)  
  tokens_tensor1 = torch.tensor([indexed_tokens1])
  segments_tensors1 = torch.tensor([segments_ids1])
  with torch.no_grad():
    # Because we set 'output_hidden_states = True', the third item will be the
    # hidden states from all layers.
    outputs1 = model(tokens_tensor1, segments_tensors1)
    hidden_states1 = outputs1[2]
  
  # shape: (25,1,17,1024) - (layers,batch size,sentence length,embedding size)
  token_embeddings1 = torch.stack(hidden_states1, dim=0)
  # Remove dimension 1, the "batches".
  # shape: (25,17,1024) - (layers,sentence length,embedding size)
  token_embeddings1 = torch.squeeze(token_embeddings1, dim=1)
  # Swap dimensions 0 and 1.（“layers” and “tokens” dimensions）
  # shape: (17,25,1024) - (sentence length,layers,embedding size)
  token_embeddings1 = token_embeddings1.permute(1, 0, 2)
  token_vecs_sum1 = []

  # shape: (17,1024)
  for token in token_embeddings1:
      # Sum the vectors from the last four layers.
      sum_vec1 = torch.sum(token[-4:], dim=0)
      token_vecs_sum1.append(sum_vec1)
  
  
  # k_emb += token_vecs_sum1
  # knowledge embedding for one question
  # shape: (27,1024)
  k_emb = np.array([t.numpy() for t in token_vecs_sum1])
  # save
  np.save(os.path.join(path, str(ques_id)+'.npy'), k_emb)
  print("saved file: ", ques_id)
print("saved all")

流式输出内容被截断，只能显示最后 5000 行内容。
saved file:  27787
saved file:  27788
saved file:  27789
saved file:  27790
saved file:  27791
saved file:  27792
saved file:  27793
saved file:  27794
saved file:  27795
saved file:  27796
saved file:  27797
saved file:  27798
saved file:  27799
saved file:  27800
saved file:  27801
saved file:  27802
saved file:  27803
saved file:  27804
saved file:  27805
saved file:  27806
saved file:  27807
saved file:  27808
saved file:  27809
saved file:  27810
saved file:  27811
saved file:  27812
saved file:  27813
saved file:  27814
saved file:  27815
saved file:  27816
saved file:  27817
saved file:  27818
saved file:  27819
saved file:  27820
saved file:  27821
saved file:  27822
saved file:  27823
saved file:  27824
saved file:  27825
saved file:  27826
saved file:  27827
saved file:  27828
saved file:  27829
saved file:  27830
saved file:  27831
saved file:  27832
saved file:  27833
saved file:  27834
saved file:  27835
saved file:  27836
saved file:  27837
save

In [ ]:
!zip -r slakekg_embs.zip knowledge_embeddings/

流式输出内容被截断，只能显示最后 5000 行内容。
  adding: knowledge_embeddings/7633.npy (deflated 7%)
  adding: knowledge_embeddings/24316.npy (deflated 7%)
  adding: knowledge_embeddings/30909.npy (deflated 7%)
  adding: knowledge_embeddings/1561.npy (deflated 7%)
  adding: knowledge_embeddings/7139.npy (deflated 7%)
  adding: knowledge_embeddings/15170.npy (deflated 7%)
  adding: knowledge_embeddings/29818.npy (deflated 7%)
  adding: knowledge_embeddings/29630.npy (deflated 7%)
  adding: knowledge_embeddings/2125.npy (deflated 7%)
  adding: knowledge_embeddings/17638.npy (deflated 7%)
  adding: knowledge_embeddings/11877.npy (deflated 7%)
  adding: knowledge_embeddings/9619.npy (deflated 7%)
  adding: knowledge_embeddings/720.npy (deflated 7%)
  adding: knowledge_embeddings/5906.npy (deflated 7%)
  adding: knowledge_embeddings/21909.npy (deflated 7%)
  adding: knowledge_embeddings/27130.npy (deflated 7%)
  adding: knowledge_embeddings/8210.npy (deflated 7%)
  adding: knowledge_embeddings/11494.npy (defla

In [ ]:
!cp /content/slakekg_embs.zip /content/drive/MyDrive/data